# Web Scraper
- Beautiful Soup: https://realpython.com/beautiful-soup-web-scraper-python/

In [1]:
import pandas as pd
import time, datetime
import multiprocessing as mp
from datetime import datetime as dt
import math

## Load Data

In [2]:
bad_cols = [0,2,6,15,16,17,18,19,20,21,22,24,25,26] # unnecessary columns from example_data notebook
cols = list(range(0,26)) # list of all columns
good_cols = [i for i in cols if i not in bad_cols] # create list of useful columns
good_cols = [str(x) for x in good_cols] # convert to string
good_cols

['1', '3', '4', '5', '7', '8', '9', '10', '11', '12', '13', '14', '23']

In [4]:
start = time.time()
# read data in chunks of 100k rows at a time
chunk = pd.read_csv('../00-data/gdelt_data/2021/gdelt_sum_dec2021.csv.zip', 
                    chunksize=100000, 
                    dtype={'3': 'str'})#, # mixed data types
                    #index_col=[0]) # specify index column
# iter_csv = pd.read_csv('file.csv', iterator=True, chunksize=1000)
# df = pd.concat([chunk[chunk['field'] > constant] for chunk in iter_csv])
df = pd.concat(chunk) # concat chunks together
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")

In [ ]:
df

In [ ]:
type(df['1'][0])

In [ ]:
bad_rows = df[~df['1'].str.get(0).isin(['2'])].index # remove rows with dates that don't start with 2
bad_rows = bad_rows.append(df[~df['1'].str.get(1).isin(['0'])].index) # remove rows with dates second number isn't 0
df = df.drop(index = bad_rows)
df['1'] = [int(float(i)) for i in df['1']]
df

In [ ]:
filter_df = df[df['3']=='yahoo.com']
filter_df

In [ ]:
filter_urls = filter_df['4']
filter_urls

In [ ]:
filter_urls.iloc[1]

## Beautiful Soup

In [6]:
from bs4 import BeautifulSoup
import requests

URL = filter_urls.iloc[1]
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
master = soup.find(id="Masterwrap2Col")
print(master.prettify())

In [ ]:
final = master.find("div", class_="caas-body")
print(final.text)

In [ ]:
sum_urls = filter_urls[0:10]
sum_urls

In [ ]:
from bs4 import BeautifulSoup
import requests
ls = []
for i in range(len(sum_urls)): #filter_urls
    URL = sum_urls.iloc[i] # filter_urls
    page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        try:
            master = soup.find(id="Masterwrap")
            final = master.find("div", class_="caas-body")
        except AttributeError:
            master = soup.find(id="Masterwrap2Col")
            final = master.find("div", class_="caas-body")
    except AttributeError:
        ls.append(URL)
        pass

In [ ]:
ls

In [ ]:
filter_urls.iloc[0]

In [7]:
URL = "https://finance.yahoo.com/video/tesla-amazon-alibaba-among-biggest-194513840.html"#filter_urls.iloc[0]
page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
soup = BeautifulSoup(page.content, "html.parser")
try:
    try:
        master = soup.find(id="Masterwrap")
        final = master.find("div", class_="caas-body")
    except AttributeError:
        master = soup.find(id="Masterwrap2Col")
        final = master.find("div", class_="caas-body")
except AttributeError:
    ls.append(URL)
    pass
final.text

'Stocks continue their sell-off, with Tesla, Amazon, and Alibaba among the biggest decliners.Video Transcript[MUSIC PLAYING]- A massive sell-off with just over 25 minutes-- 35 minutes to go in the trading day. Dow still off just over 1,300 points. S&P and NASDAQ firmly in the red.Taking a look at some of the trending tickers on our site. The biggest decliners that we are seeing. Tesla off just nearly 10%. Amazon, Alibaba, Aurora, and Apple rounding out the top five trending tickers, biggest decliners, on our site.'